# 스태킹
- 개별적인 여러 알고리즘을 서로 결합해 예측 결과를 도출
- 개별 알고리즘으로 예측한 데이터를 기반으로 다시 예측 수행
    - 이런 방식을 메타 모델이라고 함
- 여러 개의 모델에 대한 예측값을 합한 후(스태킹) 이에 대한 예측을 다시 수행

## 기본 예시

In [1]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

In [2]:
# 개별 ML 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 스태킹으로 만들어진 데이터 셋 학습, 예측할 최종 모델
lr_final = LogisticRegression(C=10)

# 개별 모델 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [3]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print(f'KNN 정확도 : {accuracy_score(y_test, knn_pred):.4f}')
print(f'랜덤 포레스트 정확도 : {accuracy_score(y_test, rf_pred):.4f}')
print(f'결정 트리 정확도 : {accuracy_score(y_test, dt_pred):.4f}')
print(f'에이다부스트 정확도 : {accuracy_score(y_test, ada_pred):.4f}')

KNN 정확도 : 0.9211
랜덤 포레스트 정확도 : 0.9649
결정 트리 정확도 : 0.9123
에이다부스트 정확도 : 0.9561


In [4]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

# transpose
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [5]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print(f'최종 메타 모델의 예측 정확도 : {accuracy_score(y_test, final):.4f}')

최종 메타 모델의 예측 정확도 : 0.9737


## CV 세트 기반의 스태킹
- 과적합을 개선하기 위해 최종 메타 모델을 위한 데이터 셋을 만들 때, 교차 검증 기반으로 예측된 결과 데이터 셋 이용

### 스텝1
- K 폴드로 K번 학습-예측한 예측 결과값을 스태킹하여 저장하여 최종 메타 모델의 학습 데이터로 사용
- 각 검증마다 학습한 모델을 테스트 셋으로 예측한 결괏값을 평균내어 최종 메타 모델의 테스트 데이터로 사용

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 최종 메타 모델이 사용할 학습/테스트 데이터 생성
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    # 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('\t폴드 세트 :', folder_counter, '시작')
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)
        
        # 폴드로 나눠진 검증 데이터로 예측 후 결과 저장
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        
        # 원본 테스트 데이터로 예측 후 결과 저장
        test_pred[:, folder_counter] = model.predict(X_test_n)
    
    # 원본 테스트 데이터 예측된 결과를 평균하여 최종 메타 모델의 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    return train_fold_pred, test_pred_mean

In [12]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier model 시작
	폴드 세트 : 0 시작
	폴드 세트 : 1 시작
	폴드 세트 : 2 시작
	폴드 세트 : 3 시작
	폴드 세트 : 4 시작
	폴드 세트 : 5 시작
	폴드 세트 : 6 시작
RandomForestClassifier model 시작
	폴드 세트 : 0 시작
	폴드 세트 : 1 시작
	폴드 세트 : 2 시작
	폴드 세트 : 3 시작
	폴드 세트 : 4 시작
	폴드 세트 : 5 시작
	폴드 세트 : 6 시작
DecisionTreeClassifier model 시작
	폴드 세트 : 0 시작
	폴드 세트 : 1 시작
	폴드 세트 : 2 시작
	폴드 세트 : 3 시작
	폴드 세트 : 4 시작
	폴드 세트 : 5 시작
	폴드 세트 : 6 시작
AdaBoostClassifier model 시작
	폴드 세트 : 0 시작
	폴드 세트 : 1 시작
	폴드 세트 : 2 시작
	폴드 세트 : 3 시작
	폴드 세트 : 4 시작
	폴드 세트 : 5 시작
	폴드 세트 : 6 시작


### 스텝2
- 각 ML 모델로 만들어진 데이터 셋을 합치기

In [13]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 feature 데이터 :', X_train.shape, '원본 테스트 feature 데이터 :', X_test.shape)
print('스태킹 학습 feature 데이터 :', Stack_final_X_train.shape, '스태킹 테스트 feature 데이터 :', Stack_final_X_test.shape)

원본 학습 feature 데이터 : (455, 30) 원본 테스트 feature 데이터 : (114, 30)
스태킹 학습 feature 데이터 : (455, 4) 스태킹 테스트 feature 데이터 : (114, 4)


In [15]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print(f'최종 메타 모델의 예측 정확도 : {accuracy_score(y_test, stack_final):.4f}')

최종 메타 모델의 예측 정확도 : 0.9737
